# Package a JSL Model for listing on AWS Marketplace

This notebook provides scripts you can use to package and verify your JSL model for listing on AWS Marketplace

### Pre-requisites

1. Before you start building an ML model, you are strongly recommended watching this video to understand the overall end-to-end ML model building and listing process.
2. You need to add the managed policy AmazonEC2ContainerRegistryFullAccess to the role associated with your notebook instance.
3. In order to create listings on AWS Marketplace you will need to register an AWS account to be a seller account by following the seller registration process. This guide assumes that the notebook is to be run in the registered seller account.

### Table of Contents
1. [Step 1 - Push Model To S3](#step1)
2. [Step 2 - Prepare a Docker image](#step2)
    - [Step 2.1 -  Step 2.1 Build a Docker Image](#step21)
    - [Step 2.2 -  Test the Docker Image](#step22)
3. [Step 3 - Push the Docker Image into Amazon ECR](#step3)
4. [Step 4 - Create an ML Model Package](#step4)
5. [Step 5 - Validate model in Amazon SageMaker environment](#step5)
   - [Step 5.1 Validate Real-time inference via Amazon SageMaker Endpoint](#step51)
   - [Step 5.2 Validate batch inference via batch transform job](#step52)
6. [Step 6 - List ML model on AWS Marketplace](#step6)


Here we import all of the libraries needed throughout the notebook to complete the model packaging process. We also create the clients necessary to interact with the various services needed (e.g., ECR, SageMaker, and S3).


In [7]:
import docker
import time
import base64
import requests
import boto3
import sagemaker as sage
from sagemaker import get_execution_role, ModelPackage
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)



# Common variables
session = sage.Session()
s3_bucket = session.default_bucket()
region = session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [8]:
model_to_package = (
    "jsl-medical-translation-en-fr"
)

model_description = "Medical text translation between English (EN) and french (FR). The model supports a maximum input length of 1024 tokens."

In [ ]:
version=None

<a id="step1"><h2>Step 1: Push Model To S3</h2></a>

#### Get JSL checkpoints (both direction)

In [9]:
pip install -q gdown peft

Note: you may need to restart the kernel to use updated packages.


In [ ]:
## Get EN -> FR checkpoint
! gdown --id 1YbGWzMmZDlTH5MuK-Aq48uHxjIVF3NKb

## Get FR -> EN checkpoint
! gdown --id 1RoZpgq_s-stCG--h3qF99o-uJkN2G-GO

In [ ]:
## Unzip the checkpoints to directories for loading

! unzip en-fr_checkpoint.zip -d checkpoint_en_fr
! unzip fr-en_checkpoint.zip -d checkpoint_fr_en

!rm en-fr_checkpoint.zip
!rm fr-en_checkpoint.zip

In [12]:
from peft import PeftModel
from transformers import M2M100Tokenizer, AutoModelForSeq2SeqLM

base_model_path = "facebook/m2m100_1.2B"
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_path)
base_model.save_pretrained("base_model")

config.json:   0%|          | 0.00/909 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


#### Create tar.gz file

In [ ]:
import tarfile
import os
def create_tar_archive(artifacts, output_filename):

    with tarfile.open(output_filename, "w:gz") as tar:
        for artifact in artifacts:
            tar.add(artifact, arcname=os.path.basename(artifact))

# Package model artifacts into a single tar archive
tar_filename = f"{model_to_package}.tar.gz"
create_tar_archive(["base_model","checkpoint_en_fr", "checkpoint_fr_en"], tar_filename)

#### Upload the model artifacts to S3

In [14]:
compressed_model_artifacts_path = f'{model_to_package}.tar.gz'
model_artifacts_s3_path = f"s3://{s3_bucket}/Model-Package/{model_to_package}.tar.gz"

In [ ]:
# Upload the model artifacts to S3
s3_client.upload_file(compressed_model_artifacts_path, s3_bucket, f"Model-Package/{model_to_package}.tar.gz")
model_artifacts_s3_path

<a id="step2"><h2>Step 2: Prepare a Docker image</h2></a>

We are going to Prepare a Docker image.

<a id="step21"> Step 2.1 Build a Docker Image with johnsnowlabs library<a>

In [84]:
docker_client = docker.from_env()

In [165]:
import os
import subprocess


os.environ["DOCKER_BUILDKIT"] = "1"

docker_build_command = f"docker build --rm=true --no-cache ../docker -t {model_to_package}"

raw_build_result = subprocess.check_output(docker_build_command, 
                                           shell=True, stderr=subprocess.STDOUT
                                           )

<a id="step22">Step 2.2: Test the Docker Image<a>

In [ ]:
!docker images

In [171]:
import os
import docker
import time

def run_docker_container(model_to_package, model_base="base_model", checkpoint_en_fr="checkpoint_en_fr", checkpoint_fr_en="checkpoint_fr_en", port=8080):
    SECONDS = 1000000000  # One second in nanoseconds

    # Define the volume mappings for the models
    volumes = {
        f"{os.getcwd()}/{model_base}/": {"bind": f"/opt/ml/model/{model_base}", "mode": "rw"},
        f"{os.getcwd()}/{checkpoint_en_fr}/": {"bind": f"/opt/ml/model/{checkpoint_en_fr}", "mode": "rw"},
        f"{os.getcwd()}/{checkpoint_fr_en}/": {"bind": f"/opt/ml/model/{checkpoint_fr_en}", "mode": "rw"},
    }

    container = docker_client.containers.run(
        model_to_package,
        detach=True,
        name=model_to_package,
        command="serve",
        healthcheck={
            "test": f"curl -f http://localhost:{port}/ping || exit 1",
            "interval": 1 * SECONDS,  # One second
            "timeout": 1 * SECONDS,  # One second
        },
        ports={f"{port}/tcp": port},
        volumes=volumes,  # Mount host directories to the container directories
    )

    return container


def wait_for_container_health(container):
    # Wait until the server is ready
    while docker_client.api.inspect_container(container.name)["State"]["Health"]["Status"] != "healthy":
        print("Waiting for server to become ready...")
        time.sleep(1)
        container.reload()
        print(f"Container status: {docker_client.api.inspect_container(container.name)['State']['Health']['Status']}")

    print("Server is ready!")

In [ ]:
port=8080
container_invocation_url = f"http://127.0.0.1:{port}/invocations"

container = run_docker_container(model_to_package=model_to_package)
wait_for_container_health(container)

### Initial setup

In [183]:
english_text = "They must be distinguished from haemolytic anaemia which commonly occurs where there is elevated serum free copper in uncontrolled Wilson's disease."
english_records = [
    "Patients with chronic obstructive pulmonary disease (COPD) should be monitored for exacerbations.",
    "The patient presented with severe abdominal pain and was diagnosed with acute pancreatitis."
]


french_text = "L'administration du bromure d'uméclidinium aux femmes enceintes ne doit être envisagée que si les bénéfices attendus pour la mère justifient le risque éventuel pour le fœtus."
french_records = [
    "Les symptômes de la grippe peuvent inclure de la fièvre, des frissons, des douleurs musculaires et une toux sèche.",
    "Les patients diabétiques doivent surveiller régulièrement leur glycémie pour éviter les complications.",
]

### JSON

##### Example: 1

In [184]:
input_json = {"text": english_text, "direction": "en_to_fr"}

r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/json", "Accept": "application/json"},
    json=input_json,
)

print(r)

<Response [200]>


In [185]:
data = r.json()

pd.DataFrame(data)

,predictions
0,Ils doivent être distingués de l'anémie hémolytique qui survient fréquemment lorsqu'il y a une augmentation du taux de cuivre libre sérique dans la maladie de Wilson non contrôlée.


##### Example: 2

In [186]:
input_json = {"text": english_records, "direction": "en_to_fr"}

r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/json", "Accept": "application/json"},
    json=input_json,
)

print(r)

<Response [200]>


In [187]:
data = r.json()
pd.DataFrame(data)

,predictions
0,Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations.
1,Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë.


##### Example: 3

In [188]:
input_json = {"text": french_text, "direction": "fr_to_en"}

r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/json", "Accept": "application/json"},
    json=input_json,
)

print(r)

<Response [200]>


In [189]:
data = r.json()
pd.DataFrame(data)

,predictions
0,The administration of umeclidinium bromide to pregnant women should only be considered if the expected benefit to the mother justifies the potential risk to the foetus.


##### Example: 4

In [190]:
input_json = {"text": french_records, "direction": "fr_to_en"}

r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/json", "Accept": "application/json"},
    json=input_json,
)

print(r)

<Response [200]>


In [191]:
data = r.json()
pd.DataFrame(data)

,predictions
0,"Flu symptoms may include fever, chills, muscle pain and dry cough."
1,Diabetic patients should monitor their blood sugar levels regularly to avoid complications.


#### JSON Lines

In [220]:
import json

def create_jsonl(records, direction):
    if isinstance(records, str):
        records = [records]
    json_lines = '\n'.join(json.dumps({"text": text, "direction": direction}) for text in records)

    return json_lines


##### Example: 1

In [194]:
input_jsonl_data  = create_jsonl(english_records, direction= "en_to_fr")

r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/jsonlines",  "Accept": "application/jsonlines"},
    data=input_jsonl_data,
)

print(r)

<Response [200]>


In [195]:
print(r.text)

{"predictions": "Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations."}
{"predictions": "Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë."}


##### Example: 2

In [196]:
input_jsonl_data  = create_jsonl(french_records, direction= "fr_to_en")

r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/jsonlines",  "Accept": "application/jsonlines"},
    data=input_jsonl_data,
)

print(r)

<Response [200]>


In [197]:
print(r.text)

{"predictions": "Flu symptoms may include fever, chills, muscle pain and dry cough."}
{"predictions": "Diabetic patients should monitor their blood sugar levels regularly to avoid complications."}


#### JSON Lines with different params

In [203]:
# Combine the JSONL strings
english_jsonl = create_jsonl(english_records, direction="en_to_fr")
french_jsonl = create_jsonl(french_records, direction="fr_to_en")

input_jsonl_data = f"{english_jsonl}\n{french_jsonl}"


r = requests.post(
    container_invocation_url,
    headers={"Content-Type": "application/jsonlines",  "Accept": "application/jsonlines"},
    data=input_jsonl_data,
)

print(r)

<Response [200]>


In [204]:
print(r.text)

{"predictions": "Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations."}
{"predictions": "Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë."}
{"predictions": "Flu symptoms may include fever, chills, muscle pain and dry cough."}
{"predictions": "Diabetic patients should monitor their blood sugar levels regularly to avoid complications."}


In [ ]:
# logs
print(container.logs().decode("utf-8"))

In [206]:
# remove the container
container.stop()
container.remove()

<a id="step3"> <h2>Step 3: Push the Docker Image into Amazon ECR <a>


In [209]:
image_name = f"jsl-marketplace/{model_to_package}"

In [210]:
docker_image_arn = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}"
docker_image_arn

'189352970232.dkr.ecr.us-east-1.amazonaws.com/jsl-marketplace/jsl-medical-translation-en-fr'

The following code shows how to build the container image and push the container image to ECR using the Docker python SDK.

This code looks for an ECR repository in the account you're using and the current default region (if you're using an Amazon SageMaker notebook instance, this will be the region where the notebook instance was created). If the repository doesn't exist, the script will create it.


In [211]:
repo_exists = image_name in [
    repo["repositoryName"] for repo in ecr.describe_repositories().get("repositories")
]

if not repo_exists:
    ecr.create_repository(repositoryName=image_name)

In [212]:
ecr_auth_data = ecr.get_authorization_token()["authorizationData"][0]
username, password = (
    base64.b64decode(ecr_auth_data["authorizationToken"]).decode("utf-8").split(":")
)

docker_client.api.tag(model_to_package, docker_image_arn, tag="latest")
status = docker_client.api.push(
    docker_image_arn,
    tag="latest",
    auth_config={"username": username, "password": password},
)

<a id="step4"><h2>Step 4: Create an ML Model Package </a></h2>
In this section, we will package our artifacts (ECR Image and the trained artifact) into a Model Package. Once completed, we can list our product as a pretrained model in AWS Marketplace.


### Define parameters

In [213]:
supported_content_types = ["application/json", "application/jsonlines"]
supported_response_MIME_types = ["application/json", "application/jsonlines"]

A Model Package creation process requires you to specify following:

Docker image
Model artifacts

- You can either package these inside the docker image, as we have done in this example, or provide them as a gzipped tarball.
  Validation specification
  In order to provide confidence to sellers (and buyers) that the products work in Amazon SageMaker, before listing them on AWS Marketplace SageMaker needs to perform basic validations. The product can be listed in AWS Marketplace only if this validation process succeeds. This validation process uses the validation profile and sample data provided by you to create a transform job in your account using the Model to verify your inference image works with SageMaker.

Next, you need to identify the right instance-sizes for your ML models. You can do so by running performance tests on top of your ML Model. A [sample notebook](https://github.com/aws-samples/aws-marketplace-machine-learning/blob/master/right_size_your_sagemaker_endpoints/Right-sizing%20your%20Amazon%20SageMaker%20Endpoints.ipynb) is available to identify minimum suggested instance types.

NOTE: In addition to tuning, take into account the requirements of your model when identifying instance types. If your model does not use GPU resources, then do not include GPU instance types. Similarly, if your model does use GPU resources, but can only make use of a single GPU, do not include instance types that have multiple GPUs as it will lead to increased infrastructure charges for your customers with no performance benefit.


In [240]:
supported_realtime_inference_instance_types = ["ml.g4dn.xlarge", "ml.g5.2xlarge"]
supported_batch_transform_instance_types = ["ml.g4dn.xlarge", "ml.g5.2xlarge"]

In [241]:
if version:
    model_package_name = model_to_package.replace('.', '-').replace('_', '-') + version
else:
    model_package_name = model_to_package.replace('.', '-').replace('_', '-')
model_package_name

'jsl-medical-translation-en-fr'

In [242]:
import json
import os


en_validation_json_file = "input1.json"
fr_validation_json_file = "input2.json"

validation_input_json_path = f"s3://{s3_bucket}/{model_package_name}/validation-input-json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_package_name}/validation-output-json/"

en_validation_jsonl_file = "input1.jsonl"
fr_validation_jsonl_file = "input2.jsonl"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_package_name}/validation-input-jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_package_name}/validation-output-jsonl/"


def write_and_upload_to_s3(input_data, file_name):

    file_format = os.path.splitext(file_name)[1].lower()
    s3_key = f"{model_package_name}/validation-input-{file_format[1:]}/{file_name}"

    if file_format == ".json":
        input_data_json = json.dumps(input_data, indent=4, ensure_ascii=False)

        with open(file_name, "w", encoding='utf-8') as f:
            f.write(input_data_json)
    elif file_format == ".jsonl":
        with open(file_name, 'w', encoding='utf-8') as file:
            for line in input_data.splitlines():
                json_object = json.loads(line)
                file.write(json.dumps(json_object, ensure_ascii=False) + '\n')

    with open(file_name, 'rb') as data:
        s3_client.put_object(Bucket=s3_bucket, Key=s3_key, Body=data)

    return input_data


en_input_jsonl_data = write_and_upload_to_s3(create_jsonl(english_records, direction= "en_to_fr"), en_validation_jsonl_file)
fr_input_jsonl_data = write_and_upload_to_s3(create_jsonl(french_records, direction= "fr_to_en"), fr_validation_jsonl_file)

# Example data for JSON file
en_input_json_data = write_and_upload_to_s3({"text": english_records, "direction": "en_to_fr"}, en_validation_json_file)
fr_input_json_data = write_and_upload_to_s3({"text": french_records, "direction": "fr_to_en"}, fr_validation_json_file)


### Create Model Package

In [243]:
model_artifacts_s3_path

's3://sagemaker-us-east-1-189352970232/Model-Package/jsl-medical-translation-en-fr.tar.gz'

In [244]:
model_package = sagemaker.create_model_package(
    ModelPackageName=model_package_name,
    ModelPackageDescription=model_description,
    InferenceSpecification={
        "Containers": [
            {
                "Image": f"{docker_image_arn}:latest",
                'ModelDataUrl': model_artifacts_s3_path,
            }
        ],
        "SupportedTransformInstanceTypes": supported_batch_transform_instance_types,
        "SupportedRealtimeInferenceInstanceTypes": supported_realtime_inference_instance_types,
        "SupportedContentTypes": supported_content_types,
        "SupportedResponseMIMETypes": supported_response_MIME_types,
    },
    CertifyForMarketplace=True,  # Make sure to set this to True for Marketplace models!
    ValidationSpecification={
        "ValidationRole": role,
        "ValidationProfiles": [
            {
                "ProfileName": "Validation-test",
                "TransformJobDefinition": {
                    "BatchStrategy": "SingleRecord",
                    "TransformInput": {
                        "DataSource": {
                            "S3DataSource": {
                                "S3DataType": "S3Prefix",
                                "S3Uri": validation_input_json_path,
                            }
                        },
                        "ContentType": supported_content_types[0],
                    },
                    "TransformOutput": {
                        "S3OutputPath": validation_output_json_path,
                    },
                    "TransformResources": {
                        "InstanceType": supported_batch_transform_instance_types[0],
                        "InstanceCount": 1,
                    },
                },
            },
        ],
    },
)

In [245]:
session.wait_for_model_package(model_package_name=model_package_name)

................................................................................................................


{'ModelPackageName': 'jsl-medical-translation-en-fr',
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-1:189352970232:model-package/jsl-medical-translation-en-fr',
 'ModelPackageDescription': 'Medical text translation between English (EN) and french (FR). The model supports a maximum input length of 1024 tokens.',
 'CreationTime': datetime.datetime(2024, 8, 28, 12, 3, 48, 114000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '189352970232.dkr.ecr.us-east-1.amazonaws.com/jsl-marketplace/jsl-medical-translation-en-fr:latest',
    'ImageDigest': 'sha256:0f5651578412f9c767a42e9be187244c78a79930522746ccfc97cace055ffdfa',
    'ModelDataUrl': 's3://sagemaker-us-east-1-189352970232/Model-Package/jsl-medical-translation-en-fr.tar.gz'}],
  'SupportedTransformInstanceTypes': ['ml.m5.2xlarge',
   'ml.m4.2xlarge',
   'ml.c5.2xlarge',
   'ml.c5.4xlarge',
   'ml.g4dn.xlarge',
   'ml.g5.2xlarge'],
  'SupportedRealtimeInferenceInstanceTypes': ['ml.m5.2xlarge',
   'ml.m4.2xlarge'

Once you have executed the preceding cell, open the [Model Packages console from Amazon SageMaker](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) and check if model creation succeeded.

Choose the Model and then open the Validation tab to see the validation results.


<a id="step5"><h2>Step 5: Validate model in Amazon SageMaker environment</h2></a>

Create a deployable model from the model package

In [246]:
model = ModelPackage(
    role=role,
    model_package_arn=model_package["ModelPackageArn"],
    sagemaker_session=session,
)

<a id="step51"><h3> Step 5.1 Validate Real-time inference via Amazon SageMaker Endpoint</a>

Deploy the SageMaker model to an endpoint

In [247]:
model.deploy(
    initial_instance_count=1,
    instance_type=supported_batch_transform_instance_types[0],
    endpoint_name=model_package_name,
)
model.endpoint_name

----------!

'jsl-medical-translation-en-fr'

#### JSON

##### Example: 1

Example invocation via boto3

In [248]:
response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(en_input_json_data),
)

data = json.loads(response["Body"].read().decode("utf-8"))
pd.DataFrame(data)

,predictions
0,Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations.
1,Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë.


Example invocation via AWS CLI

In [249]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$en_validation_json_file \
    --content-type application/json \
    --accept application/json \
    --region $session.boto_region_name \
    out1.out


# Print inference
!head out1.out

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}
{"predictions": ["Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations.", "Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë."]}

##### Example: 2

Example invocation via boto3

In [250]:
response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/json",
    Accept="application/json",
    Body=json.dumps(fr_input_json_data),
)

data = json.loads(response["Body"].read().decode("utf-8"))
pd.DataFrame(data)

,predictions
0,"Flu symptoms may include fever, chills, muscle pain and dry cough."
1,Diabetic patients should monitor their blood sugar levels regularly to avoid complications.


Example invocation via AWS CLI

In [251]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$fr_validation_json_file \
    --content-type application/json \
    --accept application/json \
    --region $session.boto_region_name \
    out1.out


# Print inference
!head out1.out

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}
{"predictions": ["Flu symptoms may include fever, chills, muscle pain and dry cough.", "Diabetic patients should monitor their blood sugar levels regularly to avoid complications."]}

#### JSON Lines

##### Example: 1

Example invocation via boto3

In [252]:
response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    Body=en_input_jsonl_data
)

data = response['Body'].read().decode('utf-8')
print(data)

{"predictions": "Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations."}
{"predictions": "Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë."}


Example invocation via AWS CLI

In [253]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$en_validation_jsonl_file \
    --content-type application/jsonlines \
    --accept application/jsonlines \
    --region $session.boto_region_name \
    out2.out


# Print inference
!head out2.out

{
    "ContentType": "application/jsonlines",
    "InvokedProductionVariant": "AllTraffic"
}
{"predictions": "Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations."}
{"predictions": "Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë."}

##### Example: 2

Example invocation via boto3

In [254]:
response = sm_runtime.invoke_endpoint(
    EndpointName=model.endpoint_name,
    ContentType="application/jsonlines",
    Accept="application/jsonlines",
    Body=fr_input_jsonl_data
)

data = response['Body'].read().decode('utf-8')
print(data)

{"predictions": "Flu symptoms may include fever, chills, muscle pain and dry cough."}
{"predictions": "Diabetic patients should monitor their blood sugar levels regularly to avoid complications."}


Example invocation via AWS CLI

In [255]:
# Perform inference
!aws sagemaker-runtime invoke-endpoint \
    --endpoint-name $model.endpoint_name \
    --body fileb://$fr_validation_jsonl_file \
    --content-type application/jsonlines \
    --accept application/jsonlines \
    --region $session.boto_region_name \
    out2.out


# Print inference
!head out2.out

{
    "ContentType": "application/jsonlines",
    "InvokedProductionVariant": "AllTraffic"
}
{"predictions": "Flu symptoms may include fever, chills, muscle pain and dry cough."}
{"predictions": "Diabetic patients should monitor their blood sugar levels regularly to avoid complications."}

Clean up the endpoint and endpoint configuration

In [256]:
model.sagemaker_session.delete_endpoint(model.endpoint_name)
model.sagemaker_session.delete_endpoint_config(model.endpoint_name)

<a id="step52"><h3> Step 5.2 Validate batch inference via batch transform job</h3></a>

### JSON

Run a batch transform job

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=supported_batch_transform_instance_types[0],
    accept="application/json",
    output_path = validation_output_json_path
)
transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

Retreive results from S3

In [259]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{os.path.basename(validation_file_name)}.out"

    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)
    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data)
    display(df)

In [260]:
# EN ==> FR
process_s3_json_output_and_save(en_validation_json_file)

,predictions
0,Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations.
1,Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë.


In [261]:
# FR ==> EN
process_s3_json_output_and_save(fr_validation_json_file)

,predictions
0,"Flu symptoms may include fever, chills, muscle pain and dry cough."
1,Diabetic patients should monitor their blood sugar levels regularly to avoid complications.


### JSON Lines

Run a batch transform job

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=supported_batch_transform_instance_types[0],
    accept="application/jsonlines",
    output_path = validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

INFO:sagemaker:Creating model with name: jsl-medical-translation-en-fr-2024-08-28-12-42-42-963
INFO:sagemaker:Creating transform job with name: jsl-medical-translation-en-fr-2024-08-28-12-42-43-720


..................

Retreive results from S3

In [263]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{os.path.basename(validation_file_name)}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)


In [266]:
# EN ==> FR
process_s3_jsonlines_output_and_save(en_validation_jsonl_file)

{"predictions": "Les patients atteints d'une maladie pulmonaire obstructive chronique (POCP) doivent être surveillés en cas d'exacerbations."}
{"predictions": "Le patient présentait des douleurs abdominales sévères et a été diagnostiqué avec une pancréatite aiguë."}


In [267]:
# FR ==> EN
process_s3_jsonlines_output_and_save(fr_validation_jsonl_file)

{"predictions": "Flu symptoms may include fever, chills, muscle pain and dry cough."}
{"predictions": "Diabetic patients should monitor their blood sugar levels regularly to avoid complications."}


In [268]:
model.delete_model()

INFO:sagemaker:Deleting model with name: jsl-medical-translation-en-fr-2024-08-28-12-42-42-963


Congratulations! You just verified that the batch transform job is working as expected. Since the model is not required, you can delete it. Note that you are deleting the deployable model. Not the model package.

### <a name="step6"></a>Step 6: List ML Model on AWS Marketplace

In the [Model Packages](https://console.aws.amazon.com/sagemaker/home?region=us-east-1#/model-packages/my-resources) section of the SageMaker console you'll find the entity you created in this notebook. If it was successfully created and validated, you should be able to select the entity and choose **Publish new ML Marketplace listing**.


You will be redirected to the [AWS Marketplace Management portal](https://aws.amazon.com/marketplace/management/ml-products/) where you will be able to build a listing.

If your model targets multiple hardware types, remember to add each ModelPackage to the listing as separate versions.